In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import urllib.request as ur
import json
import datetime
%matplotlib inline

In [2]:
df = pd.read_csv('Data/zika_cleaned_data.csv', low_memory=False)

In [3]:
df.head()

,location,report_date,location_type,country,value,province,county,city
0,Argentina-Buenos_Aires,2016-03-19,province,Argentina,130,Buenos Aires,None,None
1,Argentina-Buenos_Aires,2016-03-26,province,Argentina,148,Buenos Aires,None,None
2,Argentina-Buenos_Aires,2016-04-02,province,Argentina,190,Buenos Aires,None,None
3,Argentina-Buenos_Aires,2016-04-09,province,Argentina,199,Buenos Aires,None,None
4,Argentina-Buenos_Aires,2016-04-16,province,Argentina,207,Buenos Aires,None,None


In [4]:
#pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="gtimokhina")

In [5]:
# get lat/long based on location
def get_latitude_longitude(r):
    addr = ''
    if r.city != 'None':
        addr += r.city + ', ' 
    if r.county != 'None':
        addr += r.county + ',' 
    addr +=  r.province + ',' + r.country    
    location = geolocator.geocode(addr)
    time.sleep(1)
    return str(location.latitude) + ' ' + str(location.longitude)

In [6]:
# test api for some data
location = geolocator.geocode("Buenos Aires, Argentina")
print(location.address)
print((location.latitude, location.longitude))
print(get_latitude_longitude(df.loc[0]))

Buenos Aires, Ciudad Autónoma de Buenos Aires, Argentina
(-34.6075682, -58.4370894)
-34.6075682 -58.4370894


In [7]:
# calculate lat/long for Argentina first
df_argentina = df[df['country'] == 'Argentina']
df_argentina.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 0 to 335
Data columns (total 8 columns):
location         336 non-null object
report_date      336 non-null object
location_type    336 non-null object
country          336 non-null object
value            336 non-null int64
province         336 non-null object
county           336 non-null object
city             336 non-null object
dtypes: int64(1), object(7)
memory usage: 23.6+ KB


In [8]:
# get lat/long
df_argentina['latitude_longitude'] = df_argentina.apply(lambda r: get_latitude_longitude(r), axis=1)

/Users/Gulya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
df_argentina.head()

,location,report_date,location_type,country,value,province,county,city,latitude_longitude
0,Argentina-Buenos_Aires,2016-03-19,province,Argentina,130,Buenos Aires,None,None,-34.6075682 -58.4370894
1,Argentina-Buenos_Aires,2016-03-26,province,Argentina,148,Buenos Aires,None,None,-34.6075682 -58.4370894
2,Argentina-Buenos_Aires,2016-04-02,province,Argentina,190,Buenos Aires,None,None,-34.6075682 -58.4370894
3,Argentina-Buenos_Aires,2016-04-09,province,Argentina,199,Buenos Aires,None,None,-34.6075682 -58.4370894
4,Argentina-Buenos_Aires,2016-04-16,province,Argentina,207,Buenos Aires,None,None,-34.6075682 -58.4370894


In [10]:
#create separate columns for latitude, longitude
df_argentina[['latitude','longitude']] = df_argentina['latitude_longitude'].str.split(' ', expand=True)

/Users/Gulya/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [11]:
# make lat/long columns float type
df_argentina['latitude'] = df_argentina['latitude'].astype(float)
df_argentina['longitude'] = df_argentina['longitude'].astype(float)

/Users/Gulya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Gulya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
df_argentina = df_argentina.drop(['latitude_longitude'],axis=1)  
df_argentina.head()

,location,report_date,location_type,country,value,province,county,city,latitude,longitude
0,Argentina-Buenos_Aires,2016-03-19,province,Argentina,130,Buenos Aires,None,None,-34.607568,-58.437089
1,Argentina-Buenos_Aires,2016-03-26,province,Argentina,148,Buenos Aires,None,None,-34.607568,-58.437089
2,Argentina-Buenos_Aires,2016-04-02,province,Argentina,190,Buenos Aires,None,None,-34.607568,-58.437089
3,Argentina-Buenos_Aires,2016-04-09,province,Argentina,199,Buenos Aires,None,None,-34.607568,-58.437089
4,Argentina-Buenos_Aires,2016-04-16,province,Argentina,207,Buenos Aires,None,None,-34.607568,-58.437089


In [13]:
# save csv file for future needs
df_argentina.to_csv('Argentina.csv', encoding='utf-8', index=False)